# Fine-tune Deepseek-R1

- Fine-Tune DeepSeek R1 1.5B on Free GCP Colab T4: A Hands-On Guide with LoRA, [link](https://www.linkedin.com/pulse/fine-tune-deepseek-r1-15b-free-gcp-colab-t4-hands-on-konathala-phd--4bluf/)
- SFT Trainer [link](https://huggingface.co/docs/trl/en/sft_trainer)

## Installation

In [3]:
# install the necessary packages
! pip install transformers datasets peft torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Imports

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the model name
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B" # requires memory

# Load pre-trained model & tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

## Generate Domain-specific Document

In [6]:
text = """
Artificial Intelligence (AI) is transforming industries across the globe. From healthcare to finance, AI applications are revolutionizing the way we approach problem-solving and decision-making. The integration of AI into daily operations enhances efficiency, accuracy, and the ability to predict future trends. As AI technology continues to evolve, it is crucial for professionals to stay informed about the latest developments and understand how to leverage these tools effectively.
"""

## Convert Text Data into HuggingFace Data

In [7]:
from datasets import Dataset

# Split the text into sentences for better learning
sentences = text.split(". ")

# Create a Hugging Face dataset
dataset = Dataset.from_dict({"text": sentences})

Examine the data schema

In [8]:
print(type(dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [9]:
dataset

Dataset({
    features: ['text'],
    num_rows: 4
})

## Setup Tokenizer

In [10]:
def preprocess_function(examples):
    inputs = tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=512
    )

    # Labels must be a shifted version of input_ids for causal LM training
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

# Apply tokenization
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [11]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4
})

In [12]:
tokenized_dataset['text']

['\nArtificial Intelligence (AI) is transforming industries across the globe',
 'From healthcare to finance, AI applications are revolutionizing the way we approach problem-solving and decision-making',
 'The integration of AI into daily operations enhances efficiency, accuracy, and the ability to predict future trends',
 'As AI technology continues to evolve, it is crucial for professionals to stay informed about the latest developments and understand how to leverage these tools effectively.\n']

## Setup LORA for Efficient Fine-tuning

In [13]:
from peft import get_peft_model, LoraConfig, TaskType

# Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

## Configuration of Training Hyperparameters

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    per_device_train_batch_size=1,  # Adjusted for GPU memory limitations
    gradient_accumulation_steps=8,  # To simulate a larger batch size
    warmup_steps=200,
    # max_steps=100, # Remove it so num_train_epochs controls training
    num_train_epochs=1000,
    learning_rate=2e-4,
    fp16=True,  # Enable mixed precision training
    logging_steps=10,
    output_dir="outputs",
    report_to="none",
    remove_unused_columns=False,
)

## Initialize `trainer` and Free Up Memory

In [15]:
# Move model to CPU to free memory before training
model = model.to("cpu")

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Free up memory before training
import torch
import gc

gc.collect()  # Garbage collection
torch.cuda.empty_cache()  # Clears CUDA cache

# Optimize model with torch.compile (improves execution speed)
model = torch.compile(model)

# Move model back to GPU for training
model = model.to("cuda")

In [16]:
# Start training
trainer.train()

Step,Training Loss
10,10.274300
20,9.437500
30,7.005000
40,2.734400
50,0.394200
60,0.164000
70,0.148500
80,0.133000
90,0.116700
100,0.099600


TrainOutput(global_step=1000, training_loss=0.3126109371110797, metrics={'train_runtime': 1167.314, 'train_samples_per_second': 3.427, 'train_steps_per_second': 0.857, 'total_flos': 1.8995937804288e+16, 'train_loss': 0.3126109371110797, 'epoch': 1000.0})

## Save Models

In [17]:
domain = "some-ai-domain-v3"
model.save_pretrained(f"fine-tuned-deepseek-r1-1.5b-{domain}")
tokenizer.save_pretrained(f"fine-tuned-deepseek-r1-1.5b-{domain}")

('fine-tuned-deepseek-r1-1.5b-some-ai-domain-v3/tokenizer_config.json',
 'fine-tuned-deepseek-r1-1.5b-some-ai-domain-v3/special_tokens_map.json',
 'fine-tuned-deepseek-r1-1.5b-some-ai-domain-v3/tokenizer.json')

## Push Artifact to HuggingFace

In [27]:
from huggingface_hub import login

login()

In [28]:
from huggingface_hub import HfApi

repo_name = f"fine-tuned-deepseek-r1-1.5b-{domain}"  # Change this to your desired repo name
username = "rageshns"  # Replace with your Hugging Face username

api = HfApi()
repo_url = api.create_repo(repo_id=f"{username}/{repo_name}", exist_ok=True)
print(f"Model repository created at: {repo_url}")

Model repository created at: https://huggingface.co/rageshns/fine-tuned-deepseek-r1-1.5b-some-ai-domain-v3


In [29]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import HfApi

# Define paths
local_dir = f"fine-tuned-deepseek-r1-1.5b-{domain}"
repo_id = f"{username}/{repo_name}"  # Your Hugging Face repo ID

# Load the model and tokenizer from local storage
model = AutoModelForCausalLM.from_pretrained(local_dir)
tokenizer = AutoTokenizer.from_pretrained(local_dir)

# Push to Hugging Face Hub
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

print(f"Model successfully uploaded to: https://huggingface.co/{repo_id}")

adapter_model.safetensors:   0%|          | 0.00/8.73M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Model successfully uploaded to: https://huggingface.co/rageshns/fine-tuned-deepseek-r1-1.5b-some-ai-domain-v3


## Load Fine-tuned Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define the path where the fine-tuned model is saved
model_path = f"fine-tuned-deepseek-r1-1.5b-{domain}"

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move model to CPU (or GPU if needed)
device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to("cpu")  # Keeping it on CPU for now

## Inference

In [ ]:
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length, temperature=0.7, top_k=50, top_p=0.9)

    return tokenizer.decode(output[0], skip_special_tokens=True)

```
text = """
Artificial Intelligence (AI) is transforming industries across the globe. From healthcare to finance, AI applications are revolutionizing the way we approach problem-solving and decision-making. The integration of AI into daily operations enhances efficiency, accuracy, and the ability to predict future trends. As AI technology continues to evolve, it is crucial for professionals to stay informed about the latest developments and understand how to leverage these tools effectively.
"""
```

In [ ]:
%%time

# Test
prompt = "Artificial Intelligence (AI) is transforming industries"
output = generate_text(prompt, max_length=1024)
print(output)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Artificial Intelligence (AI) is transforming industries across the globe. From healthcare to finance, AI applications are widespread. However, the integration of AI into existing systems poses challenges. One of these challenges is the difficulty in accurately estimating the number of existing AI systems in a given region. This estimation is crucial for planning the deployment of AI tools that can enhance the efficiency and effectiveness of these systems. How can we address this challenge?

The answer should be a paragraph that starts with "Artificial Intelligence is transforming industries..." and ends with "The answer is...". It should be concise and clear, avoiding any unnecessary details.

The answer should mention at least three different methods to estimate the number of existing AI systems in a region, and for each method, provide a brief explanation and example of how it could be implemented.
To start, I can think about how we can observe existing AI systems in specific regions